## ETL para analise de edições do fifa 22

In [13]:
import pandas as pd

basePath = '../tablesV2/utf/'


#### Importa dados de jogadores

In [14]:
newPlayers = basePath + 'players.txt'

dfPlayers = pd.read_csv(newPlayers, sep='\s+')
dfPlayers = dfPlayers[['firstnameid', 'lastnameid', 'commonnameid', 'playerid']].drop_duplicates(subset='playerid', keep='first')

#### Trato dados de jogadores cruzando com bases de nomes

Busco nomes de base original da EA, e gero a combinação de nome completo

In [15]:
newPlayers = basePath + 'playernames.txt'

dfPlayersName = pd.read_csv(
    newPlayers,
    sep='\t',  # Delimitador de tabulação
    encoding='utf-8',  # Codificação UTF-8, equivalente a 65001
    names=['nameid', 'commentaryid', 'name'],  # Nomes das colunas
    skiprows=1,
    engine='python'  # Usar o motor Python para maior flexibilidade
).drop_duplicates(subset='nameid', keep='first')

playersOriginalNames = dict(zip(dfPlayersName['nameid'], dfPlayersName['name']))

In [16]:
## First Name ID > 0 significa jogadores originais do fifa

dfOriginalPlayers = dfPlayers[dfPlayers['firstnameid'] > 0].copy()

colsToSearch = [('firstnameid','firstname'),('lastnameid', 'surname'), ('commonnameid','commonname')]

for col_id, col_name in colsToSearch:
    dfOriginalPlayers[col_name] = dfOriginalPlayers[col_id].map(playersOriginalNames)

dfOriginalPlayers['fullname'] = dfOriginalPlayers['firstname'] + ' ' + dfOriginalPlayers['surname']
dfOriginalPlayers.head()

,firstnameid,lastnameid,commonnameid,playerid,firstname,surname,commonname,fullname
0,15353,6531,0,27,Joe,Cole,NaN,Joe Cole
1,1882,14117,14116,41,Andrés,Iniesta Luján,Iniesta,Andrés Iniesta Luján
2,1340,45088,0,65,Alexander,NaN,NaN,NaN
3,20389,44974,0,67,Manuel,NaN,NaN,NaN
4,21082,16798,0,82,Max,Kessler,NaN,Max Kessler


Busco nome de jogadores editados 

In [17]:
editPlayersNames = basePath + 'editedplayernames.txt'
dfEditedPlayerNames = pd.read_csv(
    editPlayersNames,
    sep='\t',  # Delimitador de tabulação
    encoding='utf-8',  # Codificação UTF-8, equivalente a 65001
    names=['firstname','commonname','playerjerseyname','surname','playerid'],  # Nomes das colunas
    skiprows=1,
    engine='python'  # Usar o motor Python para maior flexibilidade
)

dfEditedPlayerNames.dropna(inplace=True)

In [18]:
## First Name ID = 0 significa jogadores editados do mod
dfEditedPlayers = dfPlayers[dfPlayers['firstnameid'] == 0]

dfEditedPlayers= dfEditedPlayers.merge(dfEditedPlayerNames[['playerid','firstname','surname','commonname']], on='playerid', how='left')
dfEditedPlayers['fullname'] = dfEditedPlayers['firstname'] + ' ' + dfEditedPlayers['surname']
dfEditedPlayerNames.head()

,firstname,commonname,playerjerseyname,surname,playerid
5,Lamine,Lamine Yamal,Lamine Yamal,Yamal,269150
6,Endrick Felipe,Endrick,Endrick,Moreira de Sousa,6357
7,Antonio,Silva,Silva,Silva,256125
8,Adrianne,Iron,Iron,Reid,1661
9,José Luis,Joselu,Joselu,Sanmartín Mato,173608


Mesclo bases de nomes editados e originais, transformando em base unica.

In [19]:
dfPlayers = pd.concat([dfOriginalPlayers,dfEditedPlayers])
dfPlayers.drop(columns=['firstnameid','lastnameid','commonnameid'],inplace=True)

### Inicia busca de times

In [20]:
teamsPath = basePath + 'teams.txt'

dfTeams = pd.read_csv(
    teamsPath,
    sep='\t',  # Delimitador de tabulação
    encoding='utf-8'# Usar o motor Python para maior flexibilidade
).drop_duplicates(subset='teamid')
dfTeams = dfTeams[['teamid', 'teamname']].dropna()

Retira seleções da listagem de times.

In [21]:
## Busco base de ligas e a base relacional de ligas e times

leaguesPath = basePath + 'leagues.txt'
leaguesTeamsPath = basePath + 'leagueteamlinks.txt'

dfLeagues = pd.read_csv(
    leaguesPath,
    usecols=['leagueid', 'leaguename'],
    sep='\t',  # Delimitador de tabulação
    encoding='utf-8'# Usar o motor Python para maior flexibilidade
).drop_duplicates(subset='leagueid')

dfLeaguesTeams = pd.read_csv(
    leaguesTeamsPath,
    usecols=['leagueid', 'teamid', 'artificialkey'],
    sep='\t',  # Delimitador de tabulação
    encoding='utf-8'# Usar o motor Python para maior flexibilidade
)

## Retiro a seleção por base do id das ligas de seleção masculina e feminina
condition = (dfLeaguesTeams['leagueid'] != 2136) & (dfLeaguesTeams['leagueid'] != 78 ) & (dfLeaguesTeams['artificialkey'] != 0) & (dfLeaguesTeams['teamid'] != 112190) & (dfLeaguesTeams['teamid'] != 111596)
dfLeaguesTeams = dfLeaguesTeams.loc[condition]

dfLeaguesTeams = dfLeaguesTeams.merge(dfLeagues, on='leagueid', how='left')
# dfLeaguesTeams.drop_duplicates(subset='teamid')
dfLeaguesTeams.head()


,leagueid,artificialkey,teamid,leaguename
0,1,1,270,Denmark Superliga (1)
1,1,2,271,Denmark Superliga (1)
2,1,3,272,Denmark Superliga (1)
3,1,4,819,Denmark Superliga (1)
4,1,5,820,Denmark Superliga (1)


In [22]:
dfteamsFull = dfTeams.merge(dfLeaguesTeams, on='teamid', how="inner")

Importo a tablea relacional entre times e jogadores

In [23]:
teamsPlayersPath = basePath + 'teamplayerlinks.txt'

dfTeamsPlayers = pd.read_csv(
    teamsPlayersPath,
    sep='\t',  # Delimitador de tabulação
    encoding='utf-8'# Usar o motor Python para maior flexibilidade
)
dfTeamsPlayers = dfTeamsPlayers[['teamid','playerid','jerseynumber', 'position']].dropna()

dfTeamsPlayers.head()

,teamid,playerid,jerseynumber,position
0,114815,27,20,29
1,101146,41,8,18
2,111674,65,18,28
3,982,67,4,3
4,982,82,19,29


In [ ]:
dfNewTeamsPlayers = dfTeamsPlayers.merge(dfteamsFull, on='teamid', how='right')
dfNewTeamsPlayers = dfNewTeamsPlayers.merge(dfPlayers,on='playerid', how='left')
dfNewTeamsPlayers = dfNewTeamsPlayers.query('playerid > 0')
# dfNewTeamsPlayers = dfNewTeamsPlayers.merge(dfTeams,on='teamid', how='left')
dfNewTeamsPlayers.to_csv('./result/newPlayers.csv')
# subColumn = ['playerid']
# has_duplicates = dfNewTeamsPlayers[dfNewTeamsPlayers.duplicated(subset=subColumn,keep=False)]
# has_duplicates = has_duplicates.sort_values(by=['playerid'])
# has_duplicates.to_csv('./result/duplicates.csv')
# print(has_duplicates.head())

Empty DataFrame
Columns: [teamid, playerid, jerseynumber, position, teamname, leagueid, artificialkey, leaguename, firstname, surname, commonname, fullname]
Index: []
